# Chapter 3: 3D Animations

In this chapter, we'll explore 3D animations in Manim. We'll learn how to:
1. Define a 3D scene using `ThreeDScene`.
2. Create and visualize `3D Mobjects`.
3. Use the `3D camera` (e.g., rotate the camera).
4. Rotate a function graph to form a `surface`.

We will also illustrate several examples to show the power of 3D animations in Manim.

In [3]:
from manim import *
config.media_width = "75%"
config.verbosity = "WARNING"

## 3.1 Setting Up a 3D Scene

To work in 3D, Manim provides a specialized scene class called **`ThreeDScene`**. It sets up a camera that supports 3D transformations and the ability to change the camera orientation.

Below is the example of creating a 3D scene we seen in Chapter 0.

In [ ]:
%%manim -qm demo

class demo(ThreeDScene): # 3D Scene
    def construct(self):
        cube = Cube(side_length=3, fill_opacity=1)
        sphere = Sphere(radius=3, fill_opacity=1, color=BLUE)
        self.begin_ambient_camera_rotation(rate=0.3)

        self.set_camera_orientation(phi= 45 * DEGREES, theta= 45 * DEGREES)
        # phi is the vertical angle, and theta is the horizontal angle

        self.play(Write(cube), run_time=2)

        self.wait(3)
        
        self.play(Transform(cube, sphere), run_time=2)

        self.play(Unwrite(cube), run_time=2) # Why do we unwrite the cube (not sphere)? Hint: Transform VS ReplacementTransform

Manim Community v0.19.0

Explanation:
- `self.set_camera_orientation(phi, theta)` sets the vertical (phi) and horizontal (theta) angles of the camera.
- `self.begin_ambient_camera_rotation(rate=0.3)` is a convenience method that keeps rotating the camera at a fixed rate.

You may notice at the last line of code, we unwrite the `cube` Mobject, but what we actually unwrite is `sphere`. Here lies an important point, recall this line of code:
```py
self.play(Transform(cube, sphere), run_time=2)
```
The `Transform` method did not vanish the `cube` Mobject, it just transformed it into `sphere`. So, when we call the method `self.unwrite(cube)`, what we actually unwrite is `sphere`.

## 3.2 Creating a 3D Mobject

Manim provides several built-in 3D shapes, such as `Cube`, `Sphere`, `Cone`, etc. Below is an example of drawing a red `Cube` and a green `Sphere`.

In [ ]:
%%manim -qm Show3DObjects

class Show3DObjects(ThreeDScene):
    def construct(self):
        self.set_camera_orientation(phi=70 * DEGREES, theta=45 * DEGREES)

        # Create a cube and a sphere
        cube = Cube(
            side_length=2.0,
            fill_opacity=1.0,
            fill_color=RED,
            stroke_width=0.5,
        )
        # default origin is at the center, let's shift cube to the left.
        cube.shift(LEFT*2)

        sphere = Sphere(
            radius=1.2,
            color=GREEN,
            fill_opacity=0.8,
        )
        sphere.shift(RIGHT*2)

        # Animate the creation
        self.play(Create(cube))
        self.play(Create(sphere))
        self.wait(2)

        # Now rotate the camera around to see them from another angle.
        self.move_camera(phi=60*DEGREES, theta=110*DEGREES, run_time=3)
        self.wait(1)

        self.play(FadeOut(cube), FadeOut(sphere))
        self.wait()

Explanation:
- **`Cube`** and **`Sphere`** are typical 3D Mobjects. We can set their color, opacity, stroke, etc.
- You may check out other shapes like **`Cone`**, **`Cylinder`**, **`Torus`**, etc.
- We used **`move_camera`** to revolve the camera from `(phi=70°,theta=45°)` to `(phi=60°,theta=110°)`.


It's worth mentioning that you can also put 2D Mobjects in a 3D scene.

Remark: for 3D axes movements, the commands are
- **`OUT`**: [0,0,1]
- **`IN`**: [0,0,-1]
- **`UP`**: [0,1,0]
- **`DOWN`**: [0,-1,0]
- **`LEFT`**: [-1,0,0]
- **`RIGHT`**: [1,0,0]

In [ ]:
%%manim -qm Animate2DIn3DScene

class Animate2DIn3DScene(ThreeDScene):
    def construct(self):
        self.set_camera_orientation(phi=75 * DEGREES, theta=-45 * DEGREES)
        self.begin_ambient_camera_rotation(rate=0.3)
        
        axes = ThreeDAxes(
            x_range=(-4, 4, 1),
            y_range=(-4, 4, 1),
            z_range=(-3, 3, 1),
            x_length=8,
            y_length=8,
            z_length=6,
        )
        self.add(axes)

        square = Square(side_length=2, color=BLUE, fill_opacity=0.7)
        self.add(square)
        self.wait(0.5)

        self.play(square.animate.shift(OUT * 1), run_time=2)
        self.wait(0.5)
        
        self.play(square.animate.shift(IN * 1), run_time=2)
        self.wait(0.5)

        self.play(square.animate.shift(UP * 1), run_time=2)
        self.wait(0.5)

        self.play(square.animate.shift(DOWN * 1), run_time=2)
        self.wait(0.5)

        self.play(square.animate.shift(LEFT * 1), run_time=2)
        self.wait(0.5)

        self.play(square.animate.shift(RIGHT * 1), run_time=2)
        self.wait(0.5)

Manim Community v0.19.0

The following examples are rather complicated, we use it only to demonstrate the possibility of combining value trackers and 3D Mobjects in animations.

In [ ]:
%%manim -qm ParaboloidContourScene

class ParaboloidContourScene(ThreeDScene):

    def construct(self):

        self.set_camera_orientation(phi=75 * DEGREES, theta=30 * DEGREES)


        axes_3d = ThreeDAxes(
            x_range=[-3, 3, 1],
            y_range=[-3, 3, 1],
            z_range=[0, 5, 1],
            x_length=6,
            y_length=6,
            z_length=5,
        ).shift(LEFT * 3.5)

        paraboloid = Surface(
            lambda u, v: axes_3d.c2p(
                u * np.cos(v),
                u * np.sin(v),
                u**2
            ),
            u_range=[0, np.sqrt(5)],
            v_range=[0, TAU],
            checkerboard_colors=[BLUE_D, BLUE_E],
            resolution=(15, 32)
        )

        axes_2d = Axes(
            x_range=[-3, 3, 1],
            y_range=[-3, 3, 1],
            x_length=5,
            y_length=5,
            axis_config={"include_tip": False}
        ).shift(RIGHT * 3.5)
        
        labels_2d = axes_2d.get_axis_labels(x_label="x", y_label="y")


        z_tracker = ValueTracker(4)

        cutting_plane = always_redraw(
            lambda: Surface(
                lambda u, v: np.array([u, v, z_tracker.get_value()]),
                u_range=[-3, 3],
                v_range=[-3, 3],
                resolution=(1, 1),
                fill_opacity=0.5,
                fill_color=GREEN,
            ).move_to(axes_3d.get_origin() + OUT * z_tracker.get_value())
        )

        intersection_3d = always_redraw(
            lambda: ParametricFunction(
                lambda t: axes_3d.c2p(
                    np.sqrt(z_tracker.get_value()) * np.cos(t),
                    np.sqrt(z_tracker.get_value()) * np.sin(t),
                    z_tracker.get_value(),
                ),
                t_range=[0, TAU],
                color=YELLOW,
                stroke_width=5,
            ) if z_tracker.get_value() >= 0 else VMobject()
        )
        
        contour_2d = always_redraw(
            lambda: Circle(
    
                radius = axes_2d.x_axis.get_unit_size() * np.sqrt(z_tracker.get_value()),
                color=YELLOW,
                stroke_width=5
            ).move_to(axes_2d.c2p(0, 0)) if z_tracker.get_value() >= 0 else VMobject()
        )

        self.play(Create(axes_3d), Create(paraboloid), run_time=2)
        self.add(cutting_plane, intersection_3d)
        self.wait(1)

        self.play(
            Create(axes_2d),
            Create(labels_2d)
        )
        self.add(contour_2d)
        self.wait(1)

        self.play(
            z_tracker.animate.set_value(0.01), 
            run_time=5,
            rate_func=linear
        )
        self.wait(0.5)

        self.play(
            z_tracker.animate.set_value(4.5),
            run_time=5,
            rate_func=linear
        )
        self.wait(1)
        self.wait(2)

Manim Community v0.19.0

In [ ]:
%%manim -qm EarthMoonSystemScene

class EarthMoonSystemScene(ThreeDScene):
    def construct(self):
        self.set_camera_orientation(phi=75 * DEGREES, theta=30 * DEGREES, zoom=1)
        title = Text("Revolution: Earth, Moon, and Sun").to_edge(UP)
        self.add_fixed_in_frame_mobjects(title)
        self.play(Write(title))

        sun = Sphere(center=ORIGIN, radius=0.8, resolution=(24, 24)).set_color(ORANGE)

        earth_orbit_radius = 5.0
        earth_orbit_path = Circle(radius=earth_orbit_radius, color=GREY, stroke_width=2)

        earth = Sphere(radius=0.3, resolution=(16, 16)).set_color(BLUE_D)

        moon_orbit_radius = 1.2
        moon_orbit_path = Circle(radius=moon_orbit_radius, color=DARK_GREY, stroke_width=1)
        
        moon = Sphere(radius=0.1, resolution=(12, 12)).set_color(LIGHT_GREY)

        earth_rev_angle = ValueTracker(0)
        moon_rev_angle = ValueTracker(0)

        def earth_updater(mob):
            angle = earth_rev_angle.get_value()
            new_position = np.array([
                earth_orbit_radius * np.cos(angle),
                earth_orbit_radius * np.sin(angle),
                0
            ])
            mob.move_to(new_position)

        def moon_updater(mob):
            earth_center = earth.get_center()
            angle = moon_rev_angle.get_value()
            relative_position = np.array([
                moon_orbit_radius * np.cos(angle),
                moon_orbit_radius * np.sin(angle),
                0
            ])
            
            mob.move_to(earth_center + relative_position)

        def moon_orbit_path_updater(mob):
            mob.move_to(earth.get_center())

        earth.add_updater(earth_updater)
        moon.add_updater(moon_updater)
        moon_orbit_path.add_updater(moon_orbit_path_updater)
        
        self.play(Create(sun), Create(earth_orbit_path), run_time=2)

        self.add(earth, moon, moon_orbit_path)
        self.play(FadeIn(earth), FadeIn(moon), FadeIn(moon_orbit_path))
        self.wait(0.5)

        self.play(
            earth_rev_angle.animate.set_value(2 * PI),
                moon_rev_angle.animate.set_value(12 * PI),
                rate_func=linear,
                run_time=12
        )

        earth.clear_updaters()
        moon.clear_updaters()
        moon_orbit_path.clear_updaters()
        self.wait(2)

Manim Community v0.19.0

## 3.3 Using the 3D Camera

We've already seen a bit of camera usage (like `move_camera`). 3D camera usage typically involves changing:
- The angles `phi` (vertical angle) and `theta` (horizontal rotation).
- The `zoom` or `distance` for perspective.
- The `focal distance` or orthographic vs perspective mode.

Below is an example focusing on the camera transitions alone. We'll see how to animate multiple camera perspectives in one go.

In [ ]:
%%manim -qm CameraDemo

class CameraDemo(ThreeDScene):
    def construct(self):
        # Initial camera orientation
        self.set_camera_orientation(phi=45 * DEGREES, theta=-45 * DEGREES)

        axes = ThreeDAxes()
        self.play(Create(axes))
        self.wait(1)

        # Move camera to a top-down view with move_camera (instead of camera.animate)
        self.move_camera(
                phi=80*DEGREES,
                theta=0*DEGREES,
                run_time=3
            )
        self.wait(1)

        # Pan the camera around to another angle
        self.move_camera(
                phi=60*DEGREES,
                theta=90*DEGREES,
                run_time=3
            )

        self.wait(2)
        self.play(FadeOut(axes))
        self.wait()

Manim Community v0.19.0

Explanation:
- **`ThreeDAxes`** is a quick way to create X, Y, Z axes in 3D.
- We used **`set_euler_angles`** on **`self.camera`** to do camera movements. Alternatively, `move_camera` can do a similar job with phi/theta arguments.
- This demonstrates how you can do multi-step camera transitions in a single 3D scene.

The following exmple shows how to zoom in and out using the camera.

In [ ]:
%%manim -qm CameraDistanceExample

class CameraDistanceExample(ThreeDScene):
    def construct(self):
        axes = ThreeDAxes()
        cube = Cube(side_length=2, fill_opacity=0.8, fill_color=BLUE)

        label = Text("Initial State: zoom = 1.0").to_corner(UL)
        

        self.set_camera_orientation(phi=75 * DEGREES, theta=-45 * DEGREES, zoom=1.0)
        self.add(axes, cube)
        self.add_fixed_in_frame_mobjects(label)
        self.wait(1.5)

        self.play(
            label.animate.become(Text("Zooming Out: zoom = 0.3").to_corner(UL))
        )
        
        self.move_camera(zoom=0.3, run_time=3)
        self.wait(1.5)

        self.play(
            label.animate.become(Text("Zooming In: zoom = 2.0").to_corner(UL))
        )

        self.move_camera(zoom=2.0, run_time=3)
        self.wait(1.5)
        
        self.play(
            label.animate.become(Text("Back to initial state").to_corner(UL))
        )
        
        self.move_camera(zoom=1.0, run_time=3)
        self.wait(2)

Manim Community v0.19.0

## 3.4 Rotating a Function Graph to Form a Surface

A neat technique in 3D is to **revolve** a function around an axis, forming a surface of revolution. In Manim, you can do this in a couple of ways, but one straightforward approach is to define a parametric surface with a `ParametricSurface` object.

For example, consider a function in the XY-plane:  \( z = f(x) \). If we revolve it around the X-axis, the resulting surface in 3D can be described by a parametric equation. We'll illustrate that below. We'll revolve a simple shape, like a half-circle function, around the X-axis to form a sphere-ish object, or revolve a parabola for a bowl shape.

In [13]:
%%manim -qm RevolveCurveToSurface


class RevolveCurveToSurface(ThreeDScene):
    def construct(self):
        self.set_camera_orientation(phi=70 * DEGREES, theta=-30 * DEGREES)

        axes = ThreeDAxes(
            x_range=[-1.5,1.5,1],
            y_range=[-1.5,1.5,1],
            z_range=[-1,1,1],
        )
        self.play(Create(axes))

        # The function we want to revolve around the x-axis:
        # z = sqrt(1 - x^2), x in [-1,1].
        # We'll revolve about x-axis => (x, r*sin(v), r*cos(v)) with r = sqrt(1 - x^2).
        def revolve(u, v):
            x = u
            r = np.sqrt(1 - u**2)
            y = r * np.sin(v)
            z = r * np.cos(v)
            return np.array([x, y, z])

        #  A helper function that returns a Surface covering v in [0, alpha * TAU]
        #  This way, alpha ∈ [0,1] effectively sweeps out the surface from 0 to 2π.
        def revolve_surface(alpha):
            max_v = alpha * TAU
            return Surface(
                lambda u, v: revolve(u, v),
                u_range=[-1, 1],
                v_range=[0, max_v],
                fill_opacity=0.8,
                checkerboard_colors=[BLUE_D, BLUE_E],
                resolution=(24, 48),  # increase for smoother
            )

        # We use a ValueTracker to animate alpha from 0 -> 1
        alpha_tracker = ValueTracker(0)
        surface = revolve_surface(alpha_tracker.get_value())

        # Updater: re-generate the partial surface each frame
        def update_surface(mobj):
            alpha = alpha_tracker.get_value()
            new_mobj = revolve_surface(alpha)
            mobj.become(new_mobj)

        surface.add_updater(update_surface)
        self.add(surface)
        self.wait(1)

        # Animate alpha from 0 to 1 over 5 seconds
        self.play(alpha_tracker.animate.set_value(1), run_time=5)
        self.wait(1)

        # move the camera to see the final shape from another angle
        self.move_camera(phi=60*DEGREES, theta=70*DEGREES, run_time=3)
        self.wait(2)

        # Clean up
        surface.remove_updater(update_surface)
        self.play(FadeOut(surface), FadeOut(axes))
        self.wait()

Manim Community v0.19.0

Explanation:
- We define a parametric function `param_surface(u, v)` that sweeps `u` in the horizontal axis, but rotates `z`-value around the X-axis by angle `v`.
- **`ParametricSurface`** renders the mesh with color patterns or a checkerboard pattern.
- We can revolve any function you like, or even revolve around different axes with a different param.
